# Import libraries and setup config

In [70]:
from datetime import datetime, timedelta
import os, sys, time
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set("paper")
pd.set_option('display.max_columns', None)
import gzip
import pytz
from io import StringIO

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('mode.chained_assignment',None)


In [2]:
df_raw_data = pd.read_csv(f'../../Converted_Datasets/extrasensory_dataset.csv')
df_raw_data.info()
df_raw_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380031 entries, 0 to 380030
Columns: 105 entries, uuid to With friends_missing
dtypes: bool(102), int64(1), object(2)
memory usage: 45.7+ MB


,uuid,timestamp,Lying down,Sitting,Walking,Running,Bicycling,Sleeping,Lab work,In class,In a meeting,AtWorkplace,Indoors,Outside,In a car,On a bus,Drive - I'm the driver,Drive - I'm a passenger,AtHome,AtRestaurant,Phone in pocket,Exercise,Cooking,Shopping,Strolling,Drinking (alcohol),Bathing - shower,Cleaning,Doing laundry,Washing dishes,WatchingTV,Surfing the internet,At a party,At a bar,AtBeach,Singing,Talking,Computer work,Eating,Toilet,Grooming,Dressing,At the gym,Stairs - going up,Stairs - going down,Elevator,Standing,At school,Phone in hand,Phone in bag,Phone on table,With co-workers,With friends,uuid_missing,Lying down_missing,Sitting_missing,Walking_missing,Running_missing,Bicycling_missing,Sleeping_missing,Lab work_missing,In class_missing,In a meeting_missing,AtWorkplace_missing,Indoors_missing,Outside_missing,In a car_missing,On a bus_missing,Drive - I'm the driver_missing,Drive - I'm a passenger_missing,AtHome_missing,AtRestaurant_missing,Phone in pocket_missing,Exercise_missing,Cooking_missing,Shopping_missing,Strolling_missing,Drinking (alcohol)_missing,Bathing - shower_missing,Cleaning_missing,Doing laundry_missing,Washing dishes_missing,WatchingTV_missing,Surfing the internet_missing,At a party_missing,At a bar_missing,AtBeach_missing,Singing_missing,Talking_missing,Computer work_missing,Eating_missing,Toilet_missing,Grooming_missing,Dressing_missing,At the gym_missing,Stairs - going up_missing,Stairs - going down_missing,Elevator_missing,Standing_missing,At school_missing,Phone in hand_missing,Phone in bag_missing,Phone on table_missing,With co-workers_missing,With friends_missing
0,A7599A50-24AE-46A6-8EA6-2576F1011D81,1446762297,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,A7599A50-24AE-46A6-8EA6-2576F1011D81,False,False,False,True,True,False,True,True,False,False,True,True,False,False,False,True,False,False,True,False,True,False,False,True,False,True,True,True,True,False,True,True,True,True,False,True,False,True,False,False,True,True,True,True,False,False,False,False,False,False,False
1,A7599A50-24AE-46A6-8EA6-2576F1011D81,1446762357,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,A7599A50-24AE-46A6-8EA6-2576F1011D81,False,False,False,True,True,False,True,True,False,False,True,True,False,False,False,True,False,False,True,False,True,False,False,True,False,True,True,True,True,False,True,True,True,True,False,True,False,True,False,False,True,True,True,True,False,False,False,False,False,False,False
2,A7599A50-24AE-46A6-8EA6-2576F1011D81,1446762417,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,A7599A50-24AE-46A6-8EA6-2576F1011D81,False,False,False,True,True,False,True,True,False,False,True,True,False,False,False,True,False,False,True,False,True,False,False,True,False,True,True,True,True,False,True,True,True,True,False,True,False,True,False,False,True,True,True,True,False,False,False,False,False,False,False
3,A7599A50-24AE-46A6-8EA6-2576F1011D81,1446762487,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,A7599A50-24AE-

# Get ontological labels

In [ ]:
df_raw_data.columns.tolist()

In [3]:
all_labels = ['Lying down','Sitting', 'Walking', 'Running', 'Bicycling', 'Sleeping', 'Lab work', 'In class', 'In a meeting',
 'AtWorkplace', 'Indoors', 'Outside', 'In a car', 'On a bus', "Drive - I'm the driver", "Drive - I'm a passenger",
'AtHome', 'AtRestaurant', 'Phone in pocket', 'Exercise', 'Cooking', 'Shopping', 'Strolling', 'Drinking (alcohol)',
'Bathing - shower', 'Cleaning', 'Doing laundry', 'Washing dishes', 'WatchingTV', 'Surfing the internet', 'At a party', 
'At a bar', 'AtBeach', 'Singing', 'Talking', 'Computer work', 'Eating', 'Toilet', 'Grooming', 'Dressing', 'At the gym',
 'Stairs - going up', 'Stairs - going down', 'Elevator', 'Standing', 'At school', 'Phone in hand', 'Phone in bag',
'Phone on table', 'With co-workers', 'With friends']


contexts = [
    'housework',
    'dailyroutine',
    'exercising',
    'amusement',
    'sleeping',
    'inactivity',
    'commuting',
    'havingmeal',
    'officework'
]

locations = ['AtWorkplace','AtHome','Indoors','Outside','In a car','On a bus','At a party',
             'AtRestaurant','At a bar','AtBeach','At the gym','Elevator','At school']

phone_locations = ['Phone in pocket', 'Phone in hand', 'Phone in bag','Phone on table']

activities = ['Lying down','Sitting', 'Walking', 'Running', 'Bicycling', 'Sleeping', 'Lab work', 'In class', 
              'In a meeting', "Drive - I'm the driver", "Drive - I'm a passenger", 'Exercise', 'Cooking', 
              'Shopping', 'Strolling', 'Drinking (alcohol)','Bathing - shower', 'Cleaning', 'Doing laundry', 
              'Washing dishes', 'WatchingTV', 'Surfing the internet', 'Singing', 'Talking', 'Computer work', 
              'Eating', 'Toilet', 'Grooming', 'Dressing', 'Stairs - going up', 'Stairs - going down', 'Standing', 
              'With co-workers', 'With friends']

context_activity_relation = {
    'housework':['Cooking','Cleaning','Doing laundry','Washing dishes','Standing'],
    'dailyroutine':['Bathing - shower','Toilet', 'Grooming', 'Dressing','Indoors','Standing'],
    'exercising':['Walking', 'Running', 'Bicycling','Exercise','Indoors'],
    'amusement':['Sitting', 'Shopping', 'Strolling', 'Drinking (alcohol)', 'WatchingTV','Singing','With friends','Talking'],
    'sleeping':['Lying down','Sleeping','Indoors'],
    'inactivity':['Lying down', 'Sleeping','Indoors','Sitting','Standing'],
    'commuting':[ "Drive - I'm the driver", "Drive - I'm a passenger", 'Walking', 
                 'Bicycling','Stairs - going up', 'Stairs - going down','Outside','Sitting','Standing'],
    'havingmeal':['Eating','Indoors','Sitting'],
    'officework':['In a meeting', 'With co-workers','Lab work', 'In class','Surfing the internet','Computer work','Indoors','Sitting','Talking']
}

context_location_relation = {
    'housework':['AtHome','Indoors'],
    'dailyroutine':['AtHome','Indoors'],
    'exercising':['At the gym','AtHome','Indoors'],
    'amusement':['AtHome','AtRestaurant','At a bar','AtBeach','At a party','Indoors','Outside'],
    'sleeping':['AtHome','Indoors'],
    'inactivity':['AtHome','AtWorkplace','Indoors'],
    'commuting':['In a car','On a bus','Outside'],
    'havingmeal':['AtHome','AtWorkplace','AtRestaurant','At a bar','At school','Indoors'],
    'officework':['AtWorkplace','AtHome','At school','Elevator','Indoors']
}

context_phone_location_relation = {
    'housework':['Phone in bag','Phone on table'],
    'dailyroutine':['Phone in bag','Phone on table'],
    'exercising':['Phone in pocket', 'Phone in bag'],
    'amusement':['Phone in pocket', 'Phone in hand', 'Phone in bag','Phone on table'],
    'sleeping':['Phone in bag','Phone on table'],
    'inactivity':['Phone in pocket', 'Phone in bag','Phone on table'],
    'commuting':['Phone in pocket', 'Phone in hand'],
    'havingmeal':['Phone in pocket', 'Phone in hand','Phone on table'],
    'officework':['Phone in pocket', 'Phone in hand', 'Phone in bag','Phone on table']
}


# Deterministic Rule Based Pipeline

## get context-activity/context-location/context-phone mapping

In [ ]:


activity_map= np.zeros((len(activities),len(contexts)),dtype=bool)

for act_idx,activity in enumerate(activities):
    for ctx_idx, ctx in enumerate(contexts):
        activity_map[act_idx][ctx_idx] = 1 if activity in context_activity_relation[ctx] else 0

df_activity_map = pd.DataFrame(data = activity_map, index=activities, columns = contexts)

df_activity_map.info()
print(df_activity_map.sum(axis=1))
df_activity_map

In [ ]:
# get context-location/context-location/context-phone mapping

location_map= np.zeros((len(locations),len(contexts)),dtype=bool)

for loc_idx,location in enumerate(locations):
    for ctx_idx, ctx in enumerate(contexts):
        location_map[loc_idx][ctx_idx] = 1 if location in context_location_relation[ctx] else 0

df_location_map = pd.DataFrame(data = location_map, index=locations, columns = contexts)

df_location_map.info()
print(df_location_map.sum(axis=1))
df_location_map

In [ ]:
# get context-phone_location/context-phone_location/context-phone mapping

phone_location_map= np.zeros((len(phone_locations),len(contexts)),dtype=bool)

for loc_idx,phone_location in enumerate(phone_locations):
    for ctx_idx, ctx in enumerate(contexts):
        phone_location_map[loc_idx][ctx_idx] = 1 if phone_location in context_phone_location_relation[ctx] else 0

df_phone_location_map = pd.DataFrame(data = phone_location_map, index=phone_locations, columns = contexts)

df_phone_location_map.info()
df_phone_location_map

## Derive manual rule for context assignments

In [ ]:
# get context score over any time window based on activity, location and phone_location data 
'''
Activites are clubbed over all activities in timespan, but location and phone location is constant
'''
def get_context_scores(activity_time_map, location, phone_location,w_activity=0.6, w_location=0.3,w_phone_loc=0.1):
    ctx_scores = np.zeros_like(contexts)
    activity_weights = np.sum(activity_time_map,axis=0)/activity_time_map.shape[0]
     
#     print(activity_weights)
    ctx_activity_scores = np.zeros_like(contexts,dtype=float)
    for act_idx,act_weight in enumerate(activity_weights):
#         print(activities[act_idx],act_idx, act_weight)
        ctx_activity_scores += df_activity_map.iloc[act_idx,:].values.astype(float)*act_weight / np.sum(df_activity_map.iloc[act_idx,:])
#         print(ctx_activity_scores)
    
#     print(ctx_activity_scores)
    
    ctx_location_scores = df_location_map.loc[location,:].values.astype(float) / np.sum(df_location_map.loc[location,:])
#     print(ctx_location_scores)
    ctx_phone_location_scores = df_phone_location_map.loc[phone_location,:].values.astype(float) / np.sum(df_phone_location_map.loc[phone_location,:])
#     print(ctx_phone_location_scores)
    ctx_scores = w_activity*ctx_activity_scores + w_location*ctx_location_scores + w_phone_loc*ctx_phone_location_scores
    
    
    return ctx_scores


        
        

In [ ]:
ctx_scores = get_context_scores(df_raw_data[activities].head().values, 'AtHome','Phone on table')
ctx_scores

In [ ]:
list(zip(contexts, ctx_scores))

In [ ]:
contexts[np.argmax(ctx_scores)]

In [ ]:
ac_map = df_raw_data[activities].head().values

In [ ]:
np.sum(ac_map,axis=0).astype(float),ac_map.shape[0]

In [ ]:
ac_map.shape[0]

In [ ]:
df_activity_map.iloc[0,:].values.astype(float)

In [ ]:
df_raw_data[activities].head()

## Derive contexts for one user based on timestamps

In [ ]:
window_size_in_mins = 5
sliding_window_in_mins = 1
sample_uuid = 'A7599A50-24AE-46A6-8EA6-2576F1011D81'


df_raw_data_uuid = df_raw_data[df_raw_data.uuid==sample_uuid].sort_values(by='timestamp')
df_raw_data_uuid.info()
df_raw_data_uuid.shape

In [ ]:
start_timestamp, end_timestamp = df_raw_data_uuid.timestamp.min(), df_raw_data_uuid.timestamp.max()
time_ctx_scores = []
time_ctx = []
for window_start in range(start_timestamp, end_timestamp, sliding_window_in_mins*60):
    window_end = window_start + (window_size_in_mins*60)
    df_window = df_raw_data_uuid[(df_raw_data_uuid.timestamp>=window_start) & (df_raw_data_uuid.timestamp<window_end)]
    if df_window.shape[0] <= 0:
        time_ctx_scores.append([window_start, window_end, df_window.shape[0]] + np.zeros_like(contexts).tolist())
        time_ctx.append([window_start, window_end, df_window.shape[0], 'none'])
    else:
        window_location = locations[df_window[locations].sum(axis=0).argmax()]
        window_phone_loc = phone_locations[df_window[phone_locations].sum(axis=0).argmax()]
        window_activity_map = df_window[activities].values
#         print(window_location, window_phone_loc)
#         print(window_activity_map.shape)
        window_ctx_scores = get_context_scores(window_activity_map, window_location, window_phone_loc)
#         print(window_ctx_scores)
#         print(contexts[np.argmax(window_ctx_scores)])
        time_ctx_scores.append([window_start, window_end, df_window.shape[0]] + window_ctx_scores.tolist())
        time_ctx.append([window_start, window_end, df_window.shape[0], contexts[np.argmax(window_ctx_scores)]])

df_uuid_ctx_scores = pd.DataFrame(data=time_ctx_scores, columns=['start_timestamp','end_timestamp','obs_count']+contexts)
df_uuid_ctx = pd.DataFrame(data=time_ctx, columns=['start_timestamp','end_timestamp','obs_count','context'])


In [ ]:
df_uuid_ctx

In [ ]:
df_uuid_ctx.context.value_counts()

In [ ]:
def merge_time_contexts(time_ctx_arr,user_timezone='America/Los_Angeles'): # accepts an sorted array with [start_time, end_time, ctx] as values
    time_zone = pytz.timezone(user_timezone)
    new_time_ctx_list = []
    idx = 0
    while idx < len(time_ctx_arr):
        time_ctx = time_ctx_arr[idx]
        curr_ctx = time_ctx[-1]
        start_time = time_ctx[0]
        curr_ctx_idx=idx
        for j in range(curr_ctx_idx, len(time_ctx_arr)):
            if not (time_ctx_arr[j][-1]== curr_ctx):
                j=j-1
                break
        end_time = time_ctx_arr[j][1]
        
        new_time_ctx_list.append([datetime.fromtimestamp(start_time,tz=time_zone).strftime('%Y-%m-%d_%H:%M:%d'), 
                                  datetime.fromtimestamp(end_time,tz=time_zone).strftime('%Y-%m-%d_%H:%M:%d'), 
                                  curr_ctx])
        idx = j+1
#         print(new_time_ctx_list[-1])
    return new_time_ctx_list

    

In [ ]:
merge_time_contexts(df_uuid_ctx[['start_timestamp','end_timestamp','context']].values.tolist())

In [ ]:
df_uuid_ctx[['start_timestamp','end_timestamp','context']].values.tolist()

# Context Recognition in the wild
- We need to Frequency, Relation and Continuity to represent in a feature vector of a wild dataset

# A new approach to unknown context detection

In [ ]:
df_raw_data.info()

## get activity_time trains for each user from raw data

In [83]:
sample_uuid = 'A7599A50-24AE-46A6-8EA6-2576F1011D81'
df_uuid_data = df_raw_data[df_raw_data.uuid==sample_uuid]
df_uuid_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3898 entries, 0 to 3897
Columns: 105 entries, uuid to With friends_missing
dtypes: bool(102), int64(1), object(2)
memory usage: 510.1+ KB


In [84]:
# def get_activity_train(df_time_data, activities):
# timeseries =    
df_uuid_activities = df_uuid_data[activities]
df_uuid_activities['activity_set_id'] = df_uuid_activities.apply(lambda row: "".join(['1' if xr else '0' for xr in row]),axis=1)
df_uuid_activities['timestamp'] = df_uuid_data['timestamp']                                               

In [85]:
# roll activity subset from start to end times

df_uuid_activities = df_uuid_activities[['timestamp','activity_set_id']]
df_uuid_activities['group'] = df_uuid_activities['activity_set_id'].ne(df_uuid_activities['activity_set_id'].shift()).cumsum()
df_uuid_activities


,timestamp,activity_set_id,group
0,1446762297,0100000010000000000000010000000000,1
1,1446762357,0100000010000000000000010000000000,1
2,1446762417,0100000010000000000000010000000000,1
3,1446762487,0100000010000000000000010000000000,1
4,1446762548,0100000010000000000000010000000000,1
...,...,...,...
3893,1447307669,0000000000000000000000000000000000,81
3894,1447307737,0000000000000000000000000000000000,81
3895,1447307797,0000000000000000000000000000000000,81
3896,1447307857,0000000000000000000000000000000000,81


In [86]:
df_uuid_activities.to_csv('temp_uuid_activites.csv',index=None)

In [87]:
df_activity_train = df_uuid_activities.groupby(['activity_set_id','group'],as_index=False).agg({'timestamp':['max','min','count']})
df_activity_train = df_activity_train.sort_values(by='group')
df_activity_train.columns = ['_'.join(col).strip() for col in df_activity_train.columns.values]
df_activity_train.columns

Index(['activity_set_id_', 'group_', 'timestamp_max', 'timestamp_min',
       'timestamp_count'],
      dtype='object')

In [88]:
df_activity_train['duration']= df_activity_train['timestamp_max'] - df_activity_train['timestamp_min']
df_activity_train['max_sampling_space'] = df_activity_train['duration'] / df_activity_train['timestamp_count']
df_activity_train

,activity_set_id_,group_,timestamp_max,timestamp_min,timestamp_count,duration,max_sampling_space
79,0100000010000000000000010000000000,1,1446765017,1446762297,44,2720,61.818182
28,0010000000000010000000000000000000,2,1446765694,1446765077,11,617,56.090909
17,0010000000000000000000000000000000,3,1446766533,1446765754,14,779,55.642857
0,0000000000000000000000000000000000,4,1446766571,1446766571,1,0,0.000000
6,0000000000000000000000000000000100,5,1446766733,1446766664,2,69,34.500000
...,...,...,...,...,...,...,...
34,0010000000000100000000000000000000,77,1447278353,1447273705,77,4648,60.363636
76,0100000001000000000000000000000000,78,1447281702,1447278413,56,3289,58.732143
49,0100000000000000000000000100000000,79,1447282592,1447281762,15,830,55.333333
61,0100000000000000000001000000000000,80,1447302339,1447282652,329,19687,59.838906


In [ ]:
df_activity_train.to_csv('temp_uuid_activity_train.csv',index=False)

In [ ]:
duration_diff = 60*15 # 15mins
duration_max = 60*60*3 # 3 hours
inf_duration = 60*60*60 # 60 hours
duration_class = list(range(0,duration_max+duration_diff,duration_diff))
duration_class = np.array(duration_class+[inf_duration])
df_activity_train['duration_class'] = df_activity_train.duration.apply(lambda x: np.where(duration_class>x)[0][0])
df_activity_train

In [ ]:
activity_train = df_activity_train[['activity_set_id_','duration_class']].values
activity_train

## get activity_trains final function

In [89]:
def activity_set_id_to_activities(activity_set_id,activities):
    activity_set = []
    assert(len(activities)==len(activity_set_id))
    for i in range(len(activities)):
        if activity_set_id[i]=='1':
            activity_set.append(activities[i])
    if len(activity_set)==0:
        return 'None'
    return ','.join(activity_set) 

def get_activity_train(df_time_data, activity_cols, min_duration = 1,duration_diff = 60*15,duration_max = 60*60*3, 
                       inf_duration = 60*60*60*60, time_col='timestamp'): 
    # get activity ids
    df_activities = df_time_data[activities]
    df_activities['activity_set_id'] = df_activities.apply(lambda row: "".join(['1' if xr else '0' for xr in row]),axis=1)
    df_activities['timestamp'] = df_time_data[time_col]                                               
    
    # group activity over time 
    df_activities = df_activities[['timestamp','activity_set_id']]
    df_activities['group'] = df_activities['activity_set_id'].ne(df_activities['activity_set_id'].shift()).cumsum()
    
    df_activity_train = df_activities.groupby(['activity_set_id','group'],as_index=False).agg({'timestamp':['max','min','count']})
    df_activity_train = df_activity_train.sort_values(by='group')
    df_activity_train.columns = ['_'.join(col).strip() for col in df_activity_train.columns.values]
    
    
    # create duration class
    df_activity_train['duration']= df_activity_train['timestamp_max'] - df_activity_train['timestamp_min']
    df_activity_train = df_activity_train[df_activity_train['duration']>=min_duration]
    df_activity_train['max_sampling_space'] = df_activity_train['duration'] / df_activity_train['timestamp_count']
    
    duration_class = list(range(0,duration_max+duration_diff,duration_diff))
    duration_class = np.array(duration_class+[inf_duration])
    df_activity_train['duration_class'] = df_activity_train.duration.apply(lambda x: np.where(duration_class>x)[0][0])
    
    # get final activity, duration_class train
    df_activity_train['activity_set'] =df_activity_train['activity_set_id_'].apply(lambda x: activity_set_id_to_activities(x,activities))
    activity_train = df_activity_train[['activity_set','duration_class']].values.tolist()
    activity_train = [tuple(x) for x in activity_train]

    return activity_train

    
    
    

In [90]:
sample_activity_train = get_activity_train(df_uuid_data, activities)

In [92]:
sample_activity_train

# sample_activity_train = [tuple(x) for x in sample_activity_train]

[('Sitting,In a meeting,Talking', 4),
 ('Walking,Strolling', 1),
 ('Walking', 1),
 ('Standing', 1),
 ('Sitting,Eating', 2),
 ('Walking', 2),
 ('Sitting', 1),
 ('Walking', 1),
 ('Exercise,Talking,Standing', 13),
 ('Sitting', 1),
 ('Sitting', 3),
 ('Walking,Strolling', 1),
 ('Sitting,Surfing the internet', 8),
 ('None', 1),
 ('Sitting,Surfing the internet', 4),
 ('Lying down,Sleeping', 13),
 ('Bathing - shower,Standing', 2),
 ('Grooming,Dressing,Standing', 2),
 ('Sitting,Surfing the internet,Eating', 2),
 ("Sitting,Drive - I'm the driver", 2),
 ("Sitting,Drive - I'm the driver", 2),
 ('Sitting,Eating', 2),
 ("Sitting,Drive - I'm the driver", 1),
 ('Sitting,Surfing the internet', 1),
 ('Walking', 3),
 ('Sitting', 1),
 ('Walking', 1),
 ('Sitting', 7),
 ('None', 1),
 ('Sitting,Surfing the internet', 13),
 ('Standing', 1),
 ("Sitting,Drive - I'm the driver", 1),
 ('Standing', 4),
 ("Sitting,Drive - I'm the driver", 1),
 ('Walking,Shopping', 2),
 ("Sitting,Drive - I'm the driver", 1),
 ('Eati

In [ ]:
sample_activity_train[-1][0]

       

# activity_set_id_to_activities(sample_activity_train[-1][0], activities)

In [ ]:
sample_activity_train

## Create frequency maps for activity trains

In [55]:
lag = 2

freq_dict = dict()

freq_activity_train = []

df_freq_sets = pd.Series([tuple(sample_activity_train[i:i+lag]) for i in range(len(sample_activity_train)-lag)]).value_counts()
# df_freq_sets.columns = ['activity_train','freq']
# df_freq_sets.set_index('activity_train').to_dict(orient='index')
df_freq_sets.to_dict()

{(('Walking', 1), ('None', 1)): 9,
 (('None', 1), ('Walking', 1)): 8,
 (('Walking', 1), ("Sitting,Drive - I'm a passenger", 1)): 3,
 (('Standing', 1), ('None', 2)): 2,
 (('None', 1), ("Sitting,Drive - I'm a passenger", 1)): 2,
 (('None', 1), ('Doing laundry,Standing', 1)): 2,
 (('Sitting,Computer work', 1), ('Talking,Standing', 1)): 2,
 (('Talking,Standing', 1), ('Sitting,Computer work', 1)): 2,
 (('Sitting,Computer work', 1), ('Standing', 1)): 2,
 (('Sitting', 2), ('None', 1)): 1,
 (('Sitting,Computer work', 1), ('None', 4)): 1,
 (('None', 1), ('Walking', 2)): 1,
 (("Sitting,Drive - I'm the driver", 3), ('None', 1)): 1,
 (('Walking', 1), ("Sitting,Drive - I'm the driver", 3)): 1,
 (('None', 13), ('Walking', 1)): 1,
 (('Sitting,In a meeting', 3), ('None', 13)): 1,
 (('None', 1), ('Sitting,In a meeting', 3)): 1,
 (('Sitting,Eating', 1), ('None', 1)): 1,
 (('Walking', 1), ('Sitting,Eating', 1)): 1,
 (('None', 4), ('Walking', 1)): 1,
 (('None', 4), ('Sitting,Computer work', 1)): 1,
 (('St

## Final function for frequency maps

In [76]:
def get_ordered_subset_frequencies(activity_train, lag=2):
    freq_set = pd.Series([tuple(sorted(activity_train[i:i+lag])) for i in range(len(activity_train)-lag)]).value_counts()
    return freq_set.to_dict()

def get_unordered_subset_frequencies(activity_train, lag=2):
    freq_set = pd.Series([tuple(activity_train[i:i+lag]) for i in range(len(activity_train)-lag)]).value_counts()
    return freq_set.to_dict()

    

## Create ordered freq sets on whole data


In [71]:
set_lag = 1
freq_set_1 = dict()

for uuid_ in df_raw_data.uuid.unique():
    df_uuid_data = df_raw_data[df_raw_data.uuid==uuid_]
    uuid_activity_train = get_activity_train(df_uuid_data,activities)
    freq_set_uuid = get_ordered_subset_frequencies(uuid_activity_train, lag=set_lag)
    for key in freq_set_uuid.keys():
        if str(key) not in freq_set_1.keys():
            freq_set_1[str(key)] = freq_set_uuid[key]
        else:
            freq_set_1[str(key)] += freq_set_uuid[key]
#     print("Done UUID: ", uuid_)

df_freq_set_1 = pd.DataFrame.from_dict(freq_set_1,orient='index')
df_freq_set_1.columns=['set_freq']
df_freq_set_1 = df_freq_set_1.sort_values(by='set_freq',ascending=False)
df_freq_set_1.to_dict(orient='index')

{"(('Walking', 1),)": {'set_freq': 1044},
 "(('None', 1),)": {'set_freq': 840},
 "(('Sitting', 1),)": {'set_freq': 674},
 "(('Standing', 1),)": {'set_freq': 554},
 "(('Sitting', 2),)": {'set_freq': 257},
 "(('None', 2),)": {'set_freq': 244},
 "(('Lying down,Sleeping', 13),)": {'set_freq': 203},
 "(('Walking', 2),)": {'set_freq': 152},
 "(('Sitting', 3),)": {'set_freq': 149},
 "(('None', 3),)": {'set_freq': 141},
 "(('Bicycling,Exercise', 1),)": {'set_freq': 140},
 "(('Sitting,Computer work', 1),)": {'set_freq': 134},
 "(('Lying down', 1),)": {'set_freq': 131},
 "(('Sitting,Eating', 1),)": {'set_freq': 129},
 '(("Sitting,Drive - I\'m the driver", 1),)': {'set_freq': 117},
 "(('Sitting,Computer work', 2),)": {'set_freq': 116},
 "(('Standing', 2),)": {'set_freq': 114},
 "(('None', 13),)": {'set_freq': 105},
 "(('Sitting,Toilet', 1),)": {'set_freq': 101},
 "(('Walking,Talking', 1),)": {'set_freq': 95},
 "(('Sitting,Talking', 1),)": {'set_freq': 89},
 "(('Sitting,Surfing the internet', 1),)

In [72]:
set_lag = 2
freq_set_2 = dict()

for uuid_ in df_raw_data.uuid.unique():
    df_uuid_data = df_raw_data[df_raw_data.uuid==uuid_]
    uuid_activity_train = get_activity_train(df_uuid_data,activities)
    freq_set_uuid = get_ordered_subset_frequencies(uuid_activity_train, lag=set_lag)
    for key in freq_set_uuid.keys():
        if str(key) not in freq_set_2.keys():
            freq_set_2[str(key)] = freq_set_uuid[key]
        else:
            freq_set_2[str(key)] += freq_set_uuid[key]
#     print("Done UUID: ", uuid_)

df_freq_set_2 = pd.DataFrame.from_dict(freq_set_2,orient='index')
df_freq_set_2.columns=['set_freq']
df_freq_set_2 = df_freq_set_2.sort_values(by='set_freq',ascending=False)
df_freq_set_2.to_dict(orient='index')

{"(('Sitting', 1), ('Walking', 1))": {'set_freq': 119},
 "(('Walking', 1), ('Standing', 1))": {'set_freq': 108},
 "(('Walking', 1), ('Sitting', 1))": {'set_freq': 103},
 "(('None', 1), ('Walking', 1))": {'set_freq': 98},
 "(('Walking', 1), ('None', 1))": {'set_freq': 78},
 "(('Standing', 1), ('Walking', 1))": {'set_freq': 69},
 "(('Sitting', 1), ('None', 1))": {'set_freq': 66},
 "(('None', 1), ('Sitting', 1))": {'set_freq': 61},
 "(('Sitting', 1), ('Sitting', 1))": {'set_freq': 56},
 "(('Standing', 1), ('None', 1))": {'set_freq': 56},
 "(('Sitting', 2), ('Walking', 1))": {'set_freq': 52},
 "(('Standing', 1), ('Sitting', 1))": {'set_freq': 50},
 "(('None', 1), ('Standing', 1))": {'set_freq': 47},
 "(('Sitting', 1), ('Standing', 1))": {'set_freq': 47},
 "(('None', 1), ('None', 1))": {'set_freq': 47},
 "(('Walking', 1), ('Walking', 1))": {'set_freq': 46},
 "(('Sitting', 2), ('None', 1))": {'set_freq': 31},
 '((\'Walking\', 1), ("Sitting,Drive - I\'m the driver", 1))': {'set_freq': 30},
 "

In [73]:
set_lag = 3
freq_set_3 = dict()

for uuid_ in df_raw_data.uuid.unique():
    df_uuid_data = df_raw_data[df_raw_data.uuid==uuid_]
    uuid_activity_train = get_activity_train(df_uuid_data,activities)
    freq_set_uuid = get_ordered_subset_frequencies(uuid_activity_train, lag=set_lag)
    for key in freq_set_uuid.keys():
        if str(key) not in freq_set_3.keys():
            freq_set_3[str(key)] = freq_set_uuid[key]
        else:
            freq_set_3[str(key)] += freq_set_uuid[key]
#     print("Done UUID: ", uuid_)

df_freq_set_3 = pd.DataFrame.from_dict(freq_set_3,orient='index')
df_freq_set_3.columns=['set_freq']
df_freq_set_3 = df_freq_set_3.sort_values(by='set_freq',ascending=False)
df_freq_set_3.to_dict(orient='index')

{"(('Walking', 1), ('Sitting', 1), ('Walking', 1))": {'set_freq': 27},
 "(('Walking', 1), ('Standing', 1), ('Walking', 1))": {'set_freq': 24},
 "(('Walking', 1), ('None', 1), ('Walking', 1))": {'set_freq': 18},
 "(('None', 1), ('Sitting', 1), ('None', 1))": {'set_freq': 18},
 "(('None', 1), ('Walking', 1), ('None', 1))": {'set_freq': 17},
 "(('Sitting', 1), ('Walking', 1), ('Sitting', 1))": {'set_freq': 15},
 "(('Sitting', 1), ('Walking', 1), ('Standing', 1))": {'set_freq': 13},
 '((\'Walking\', 1), ("Sitting,Drive - I\'m the driver", 2), (\'Walking\', 1))': {'set_freq': 13},
 "(('Walking', 1), ('Standing', 1), ('Sitting', 1))": {'set_freq': 13},
 "(('Walking', 1), ('Sitting', 1), ('Sitting', 1))": {'set_freq': 12},
 "(('Walking', 1), ('Sitting', 2), ('Walking', 1))": {'set_freq': 12},
 "(('Standing', 1), ('Walking', 1), ('Sitting', 1))": {'set_freq': 12},
 "(('None', 1), ('Walking', 1), ('Standing', 1))": {'set_freq': 12},
 "(('Sitting', 1), ('Sitting', 1), ('Sitting', 1))": {'set_fre

In [50]:
len(freq_set_2.keys())

10357

In [51]:

df_freq_set

,set_freq
"(('Walking', 1), ('Sitting', 1), ('Walking', 1), ('Standing', 1))",6
"(('Walking', 1), ('Standing', 1), ('Sitting', 1), ('Walking', 1))",6
"(('Walking', 1), ('Standing', 1), ('Walking', 1), ('Standing', 1))",6
"(('None', 1), ('Lying down,Sleeping', 1), ('None', 1), ('Lying down,Sleeping', 1))",6
"(('Sitting', 1), ('Lying down', 1), ('Sitting', 1), ('Sitting', 1))",5
...,...
"(('Walking', 1), ('Walking,Strolling', 3), ('Eating,Standing', 1), ('Eating,Standing', 1))",1
"(('Sitting,Talking', 2), ('Walking', 1), ('Walking,Strolling', 3), ('Eating,Standing', 1))",1
"(('Surfing the internet,Standing', 1), ('Sitting,Talking', 2), ('Walking', 1), ('Walking,Strolling', 3))",1
"(('Walking', 1), ('Surfing the internet,Standing', 1), ('Sitting,Talking', 2), ('Walking', 1))",1


In [52]:
df_freq_set.to_dict(orient='index')

{"(('Walking', 1), ('Sitting', 1), ('Walking', 1), ('Standing', 1))": {'set_freq': 6},
 "(('Walking', 1), ('Standing', 1), ('Sitting', 1), ('Walking', 1))": {'set_freq': 6},
 "(('Walking', 1), ('Standing', 1), ('Walking', 1), ('Standing', 1))": {'set_freq': 6},
 "(('None', 1), ('Lying down,Sleeping', 1), ('None', 1), ('Lying down,Sleeping', 1))": {'set_freq': 6},
 "(('Sitting', 1), ('Lying down', 1), ('Sitting', 1), ('Sitting', 1))": {'set_freq': 5},
 "(('Sitting', 1), ('Walking', 1), ('Standing', 1), ('Walking', 1))": {'set_freq': 5},
 "(('Walking', 1), ('Standing', 1), ('Walking', 1), ('Sitting', 1))": {'set_freq': 5},
 "(('Walking', 1), ('None', 1), ('Walking', 1), ('None', 1))": {'set_freq': 5},
 "(('Sitting', 1), ('Walking', 1), ('Sitting', 1), ('Walking', 1))": {'set_freq': 5},
 "(('Standing', 1), ('Walking', 1), ('Sitting', 1), ('Walking', 1))": {'set_freq': 5},
 "(('Lying down', 1), ('Sitting', 1), ('Sitting', 1), ('Sitting', 1))": {'set_freq': 4},
 "(('Walking', 1), ('Sitting'

In [75]:
eval("(('None', 3),)")

(('None', 3),)

## Create unordered freq sets on whole data


In [80]:
set_lag = 1
freq_set_1 = dict()

for uuid_ in df_raw_data.uuid.unique():
    df_uuid_data = df_raw_data[df_raw_data.uuid==uuid_]
    uuid_activity_train = get_activity_train(df_uuid_data,activities)
    freq_set_uuid = get_unordered_subset_frequencies(uuid_activity_train, lag=set_lag)
    for key in freq_set_uuid.keys():
        if str(key) not in freq_set_1.keys():
            freq_set_1[str(key)] = freq_set_uuid[key]
        else:
            freq_set_1[str(key)] += freq_set_uuid[key]
#     print("Done UUID: ", uuid_)

df_freq_set_1 = pd.DataFrame.from_dict(freq_set_1,orient='index')
df_freq_set_1.columns=['set_freq']
df_freq_set_1 = df_freq_set_1.sort_values(by='set_freq',ascending=False)
df_freq_set_1.info()
df_freq_set_1.to_dict(orient='index')

<class 'pandas.core.frame.DataFrame'>
Index: 1087 entries, (('Walking', 1),) to (('Sitting,In a meeting,With co-workers', 8),)
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   set_freq  1087 non-null   int64
dtypes: int64(1)
memory usage: 17.0+ KB


{"(('Walking', 1),)": {'set_freq': 1044},
 "(('None', 1),)": {'set_freq': 840},
 "(('Sitting', 1),)": {'set_freq': 674},
 "(('Standing', 1),)": {'set_freq': 554},
 "(('Sitting', 2),)": {'set_freq': 257},
 "(('None', 2),)": {'set_freq': 244},
 "(('Lying down,Sleeping', 13),)": {'set_freq': 203},
 "(('Walking', 2),)": {'set_freq': 152},
 "(('Sitting', 3),)": {'set_freq': 149},
 "(('None', 3),)": {'set_freq': 141},
 "(('Bicycling,Exercise', 1),)": {'set_freq': 140},
 "(('Sitting,Computer work', 1),)": {'set_freq': 134},
 "(('Lying down', 1),)": {'set_freq': 131},
 "(('Sitting,Eating', 1),)": {'set_freq': 129},
 '(("Sitting,Drive - I\'m the driver", 1),)': {'set_freq': 117},
 "(('Sitting,Computer work', 2),)": {'set_freq': 116},
 "(('Standing', 2),)": {'set_freq': 114},
 "(('None', 13),)": {'set_freq': 105},
 "(('Sitting,Toilet', 1),)": {'set_freq': 101},
 "(('Walking,Talking', 1),)": {'set_freq': 95},
 "(('Sitting,Talking', 1),)": {'set_freq': 89},
 "(('Sitting,Surfing the internet', 1),)

In [81]:
set_lag = 2
freq_set_2 = dict()

for uuid_ in df_raw_data.uuid.unique():
    df_uuid_data = df_raw_data[df_raw_data.uuid==uuid_]
    uuid_activity_train = get_activity_train(df_uuid_data,activities)
    freq_set_uuid = get_unordered_subset_frequencies(uuid_activity_train, lag=set_lag)
    for key in freq_set_uuid.keys():
        if str(key) not in freq_set_2.keys():
            freq_set_2[str(key)] = freq_set_uuid[key]
        else:
            freq_set_2[str(key)] += freq_set_uuid[key]
#     print("Done UUID: ", uuid_)

df_freq_set_2 = pd.DataFrame.from_dict(freq_set_2,orient='index')
df_freq_set_2.columns=['set_freq']
df_freq_set_2 = df_freq_set_2.sort_values(by='set_freq',ascending=False)
df_freq_set_2.info()
df_freq_set_2.to_dict(orient='index')

<class 'pandas.core.frame.DataFrame'>
Index: 6057 entries, (('Sitting', 1), ('Walking', 1)) to (('None', 1), ('Sitting,WatchingTV', 13))
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   set_freq  6057 non-null   int64
dtypes: int64(1)
memory usage: 94.6+ KB


{"(('Sitting', 1), ('Walking', 1))": {'set_freq': 119},
 "(('Walking', 1), ('Standing', 1))": {'set_freq': 108},
 "(('Walking', 1), ('Sitting', 1))": {'set_freq': 103},
 "(('None', 1), ('Walking', 1))": {'set_freq': 98},
 "(('Walking', 1), ('None', 1))": {'set_freq': 78},
 "(('Standing', 1), ('Walking', 1))": {'set_freq': 69},
 "(('Sitting', 1), ('None', 1))": {'set_freq': 66},
 "(('None', 1), ('Sitting', 1))": {'set_freq': 61},
 "(('Sitting', 1), ('Sitting', 1))": {'set_freq': 56},
 "(('Standing', 1), ('None', 1))": {'set_freq': 56},
 "(('Sitting', 2), ('Walking', 1))": {'set_freq': 52},
 "(('Standing', 1), ('Sitting', 1))": {'set_freq': 50},
 "(('None', 1), ('Standing', 1))": {'set_freq': 47},
 "(('Sitting', 1), ('Standing', 1))": {'set_freq': 47},
 "(('None', 1), ('None', 1))": {'set_freq': 47},
 "(('Walking', 1), ('Walking', 1))": {'set_freq': 46},
 "(('Sitting', 2), ('None', 1))": {'set_freq': 31},
 '((\'Walking\', 1), ("Sitting,Drive - I\'m the driver", 1))': {'set_freq': 30},
 "

In [82]:
set_lag = 3
freq_set_3 = dict()

for uuid_ in df_raw_data.uuid.unique():
    df_uuid_data = df_raw_data[df_raw_data.uuid==uuid_]
    uuid_activity_train = get_activity_train(df_uuid_data,activities)
    freq_set_uuid = get_unordered_subset_frequencies(uuid_activity_train, lag=set_lag)
    for key in freq_set_uuid.keys():
        if str(key) not in freq_set_3.keys():
            freq_set_3[str(key)] = freq_set_uuid[key]
        else:
            freq_set_3[str(key)] += freq_set_uuid[key]
#     print("Done UUID: ", uuid_)

df_freq_set_3 = pd.DataFrame.from_dict(freq_set_3,orient='index')
df_freq_set_3.columns=['set_freq']
df_freq_set_3 = df_freq_set_3.sort_values(by='set_freq',ascending=False)
df_freq_set_3.info()
df_freq_set_3.to_dict(orient='index')

<class 'pandas.core.frame.DataFrame'>
Index: 9454 entries, (('Walking', 1), ('Sitting', 1), ('Walking', 1)) to (("Sitting,Drive - I'm a passenger", 1), ('None', 4), ('Sitting', 2))
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   set_freq  9454 non-null   int64
dtypes: int64(1)
memory usage: 147.7+ KB


{"(('Walking', 1), ('Sitting', 1), ('Walking', 1))": {'set_freq': 27},
 "(('Walking', 1), ('Standing', 1), ('Walking', 1))": {'set_freq': 24},
 "(('Walking', 1), ('None', 1), ('Walking', 1))": {'set_freq': 18},
 "(('None', 1), ('Sitting', 1), ('None', 1))": {'set_freq': 18},
 "(('None', 1), ('Walking', 1), ('None', 1))": {'set_freq': 17},
 "(('Sitting', 1), ('Walking', 1), ('Sitting', 1))": {'set_freq': 15},
 "(('Sitting', 1), ('Walking', 1), ('Standing', 1))": {'set_freq': 13},
 '((\'Walking\', 1), ("Sitting,Drive - I\'m the driver", 2), (\'Walking\', 1))': {'set_freq': 13},
 "(('Walking', 1), ('Standing', 1), ('Sitting', 1))": {'set_freq': 13},
 "(('Walking', 1), ('Sitting', 1), ('Sitting', 1))": {'set_freq': 12},
 "(('Walking', 1), ('Sitting', 2), ('Walking', 1))": {'set_freq': 12},
 "(('Standing', 1), ('Walking', 1), ('Sitting', 1))": {'set_freq': 12},
 "(('None', 1), ('Walking', 1), ('Standing', 1))": {'set_freq': 12},
 "(('Sitting', 1), ('Sitting', 1), ('Sitting', 1))": {'set_fre

In [50]:
len(freq_set_2.keys())

10357

In [51]:
df_freq_set

,set_freq
"(('Walking', 1), ('Sitting', 1), ('Walking', 1), ('Standing', 1))",6
"(('Walking', 1), ('Standing', 1), ('Sitting', 1), ('Walking', 1))",6
"(('Walking', 1), ('Standing', 1), ('Walking', 1), ('Standing', 1))",6
"(('None', 1), ('Lying down,Sleeping', 1), ('None', 1), ('Lying down,Sleeping', 1))",6
"(('Sitting', 1), ('Lying down', 1), ('Sitting', 1), ('Sitting', 1))",5
...,...
"(('Walking', 1), ('Walking,Strolling', 3), ('Eating,Standing', 1), ('Eating,Standing', 1))",1
"(('Sitting,Talking', 2), ('Walking', 1), ('Walking,Strolling', 3), ('Eating,Standing', 1))",1
"(('Surfing the internet,Standing', 1), ('Sitting,Talking', 2), ('Walking', 1), ('Walking,Strolling', 3))",1
"(('Walking', 1), ('Surfing the internet,Standing', 1), ('Sitting,Talking', 2), ('Walking', 1))",1


In [52]:
df_freq_set.to_dict(orient='index')

{"(('Walking', 1), ('Sitting', 1), ('Walking', 1), ('Standing', 1))": {'set_freq': 6},
 "(('Walking', 1), ('Standing', 1), ('Sitting', 1), ('Walking', 1))": {'set_freq': 6},
 "(('Walking', 1), ('Standing', 1), ('Walking', 1), ('Standing', 1))": {'set_freq': 6},
 "(('None', 1), ('Lying down,Sleeping', 1), ('None', 1), ('Lying down,Sleeping', 1))": {'set_freq': 6},
 "(('Sitting', 1), ('Lying down', 1), ('Sitting', 1), ('Sitting', 1))": {'set_freq': 5},
 "(('Sitting', 1), ('Walking', 1), ('Standing', 1), ('Walking', 1))": {'set_freq': 5},
 "(('Walking', 1), ('Standing', 1), ('Walking', 1), ('Sitting', 1))": {'set_freq': 5},
 "(('Walking', 1), ('None', 1), ('Walking', 1), ('None', 1))": {'set_freq': 5},
 "(('Sitting', 1), ('Walking', 1), ('Sitting', 1), ('Walking', 1))": {'set_freq': 5},
 "(('Standing', 1), ('Walking', 1), ('Sitting', 1), ('Walking', 1))": {'set_freq': 5},
 "(('Lying down', 1), ('Sitting', 1), ('Sitting', 1), ('Sitting', 1))": {'set_freq': 4},
 "(('Walking', 1), ('Sitting'

In [75]:
eval("(('None', 3),)")

(('None', 3),)